#### Google Palm LLM & API key setup

In [59]:
from langchain_groq import ChatGroq
api_key = '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'
llm = ChatGroq(
    temperature=0, 
    groq_api_key= api_key, 
    model_name="llama-3.1-70b-versatile"
)

#### Connect with database and ask some basic questions

In [60]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [61]:
db_user = "root"
db_password = "password"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [62]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:Question: How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5;
SQLResult: [(39,)]
Answer:Question: How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5;
> Finished chain.


Above is the correct answer 👍🏼

In [63]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: 
SELECT SUM(`price` * `stock_quantity`) AS total_price 
FROM t_shirts 
WHERE `size` = 'S';
SQLResult: [(Decimal('16713'),)]
Answer:Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS total_price 
FROM t_shirts 
WHERE `size` = 'S';
> Finished chain.


It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names. We need to tell it somehow that price is price per unit and the actual query should be,

SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [64]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:Question: SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM t_shirts WHERE `size` = 'S'
SQLResult: [(Decimal('16713'),)]
Answer:Question: SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM t_shirts WHERE `size` = 'S'
> Finished chain.


we will use qns2 value later on in this notebook. So hold on for now and let's check another query

In [65]:
# qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")

Above, it returned a wrong query which generated an error during query execution. It thinks discount
table would have start and end date which is normally true but in our table there is no start or end date column.
One thing we can do here is run the query directly.

In [66]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:Question: 
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id

SQLQuery: 
SELECT 
  SUM(a.`total_amount` * ((100 - COALESCE(discounts.`pct_discount`, 0)) / 100)) AS `total_revenue`
FROM 
  (SELECT 
     SUM(t_shirts.`price` * t_shirts.`stock_quantity`) AS `total_amount`, 
     t_shirts.`t_shirt_id`
   FROM 
     t_shirts
   WHERE 
     t_shirts.`brand` = 'Levi'
   GROUP BY 
     t_shirts.`t_shirt_id`) a
  LEFT JOIN discounts ON a.`t_shirt

It produced a correct answer when explicit query was given. 17462 is the total revenue without discounts. The total discount is 736.6. Hence revenue post discount is 17462-736.6=16725.4

Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [67]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:Question: SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' LIMIT 5;
SQLResult: [(Decimal('36933'),)]
Answer:Question: What is the total value of Levi t-shirts in stock?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS `total_value` FROM `t_shirts` WHERE `brand` = 'Levi' LIMIT 5;
> Finished chain.


In [68]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:Question: How many white color Levi's t shirts we have available?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(98,), (79,), (69,), (86,)]
Answer:Question: How many white color Levi's t shirts we have available?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White'
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [69]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 5;
SQLResult: [(Decimal('332'),)]
Answer:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White' LIMIT 5;
> Finished chain.


#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [79]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
#     {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
#      'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
# (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
# group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
#  """,
#      'SQLResult': "Result of the SQL query",
#      'Answer': qns3} ,
#      {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
#       'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
#       'SQLResult': "Result of the SQL query",
#       'Answer' : qns4},
#     {'Question': "How many white color Levi's shirt I have?",
#      'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
#      'SQLResult': "Result of the SQL query",
#      'Answer' : qns5
#      }
]
few_shots


[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': {'query': 'How many t-shirts do we have left for nike in extra small size and white color?',
   'result': "Question: How many t-shirts do we have left for nike in extra small size and white color?\nSQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White' LIMIT 5;"}},
 {'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query',
  'Answer': "Question: SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'\nSQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM t_shirts WHERE `size` = 'S'"}]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [83]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Define the embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Prepare texts to vectorize and simplified metadata
to_vectorize = []
metadatas = []

try:
    for example in few_shots:
        # Process example text
        example_str = " ".join(
            " ".join(str(value) for value in v.values()) if isinstance(v, dict) else str(v)
            for v in example.values()
        )
        to_vectorize.append(example_str)
        
        # Simplify metadata: Convert to dictionary with simple types
        metadata_dict = {
            "Question": example.get("Question", ""),
            "SQLQuery": example.get("SQLQuery", ""),
            "SQLResult": example.get("SQLResult", "")
        }
        metadatas.append(metadata_dict)
except Exception as e:
    print(f"An error occurred while processing examples: {e}")

# Create the vector store
try:
    vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=metadatas)
except Exception as e:
    print(f"An error occurred while creating the vector store: {e}")


In [84]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{},
 {'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'}]

In [85]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [86]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [87]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [88]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [98]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)


input_variables=['input', 'table_info', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f89b8cbbd60>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Only use the following tables:\n{table_info}\n\nQuestion: {input}' prefix='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You 

In [101]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [104]:
try:
    new_chain("How many white color small Levi's in shirt I have?")
except Expection as e:
    print(e)



> Entering new SQLDatabaseChain chain...
How many white color small Levi's in shirt I have?
SQLQuery:Question: How many white color small Levi's in shirt I have?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White' AND `size` = 'S'
SQLResult: [(79,)]
Answer:Question: How many white color small Levi's in shirt I have?
SQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White' AND `size` = 'S'
> Finished chain.


Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [105]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: 
SELECT SUM(`price` * `stock_quantity`) AS total_price 
FROM t_shirts 
WHERE `size` = 'S';
SQLResult: [(Decimal('16713'),)]
Answer:Question: How much is the price of the inventory for all small size t-shirts?
SQLQuery: SELECT SUM(`price` * `stock_quantity`) AS total_price 
FROM t_shirts 
WHERE `size` = 'S';
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': "Question: How much is the price of the inventory for all small size t-shirts?\nSQLQuery: SELECT SUM(`price` * `stock_quantity`) AS total_price \nFROM t_shirts \nWHERE `size` = 'S';"}

In [ ]:
new_chain("How much is the price of all white color levi t shirts?")

In [ ]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')